
### Préparation des données (Pour ce module)

On recrée une table `ventes` simple pour bien illustrer les calculs et les tries.

In [1]:
import sqlite3

In [2]:
# Si 'analyse_ventes.db' n'existe pas, Python le crée vide.
conn = sqlite3.connect('analyse_ventes.db')

# Le curseur est notre "robot" qui va exécuter les ordres
cursor = conn.cursor()

print("Connexion établie vers 'ecole_progressive.db'")

Connexion établie vers 'ecole_progressive.db'


In [3]:
import sqlite3
import pandas as pd
import os
conn = sqlite3.connect('analyse_ventes.db')
# Création d'une table de ventes
conn.execute("""
    CREATE TABLE ventes (
        id INTEGER PRIMARY KEY,
        vendeur TEXT,
        produit TEXT,
        quantite INTEGER,
        prix_unitaire REAL,
        categorie TEXT
    )
""")

OperationalError: table ventes already exists

In [4]:
# Insertion de données variées
data = [
    ('Alice', 'Laptop', 5, 1200.0, 'High-Tech'),
    ('Bob', 'Clavier', 10, 50.0, 'Accessoire'),
    ('Alice', 'Souris', 20, 25.0, 'Accessoire'),
    ('Charlie', 'Laptop', 2, 1250.0, 'High-Tech'),
    ('Alice', 'Ecran', 4, 300.0, 'High-Tech'),
    ('Bob', 'Webcam', 5, 45.0, 'Accessoire'),
    ('David', 'Chaise', 10, 150.0, 'Mobilier'),
    ('David', 'Bureau', 2, 400.0, 'Mobilier')
]
conn.executemany("INSERT INTO ventes (vendeur, produit, quantite, prix_unitaire, categorie) VALUES (?,?,?,?,?)", data)
conn.commit()
print("Données de ventes initialisées.")

Données de ventes initialisées.


---

## 1. SELECT (La Projection)

Le choix des colonnes. Ne jamais utiliser `SELECT *` en production si vous n'avez besoin que d'une colonne (c'est du gaspillage de bande passante).

###  SELECT - Exemple 1 (Tout sélectionner)

**Cas d'usage :** Exploration initiale des données brutes.

In [7]:
# L'étoile (*) signifie "Toutes les colonnes"
sql_sel_1 = "SELECT * FROM ventes"
for x in conn.execute(sql_sel_1):
    print(x)
#display(pd.read_sql_query(sql_sel_1, conn))

(1, 'Alice', 'Laptop', 5, 1200.0, 'High-Tech')
(2, 'Bob', 'Clavier', 10, 50.0, 'Accessoire')
(3, 'Alice', 'Souris', 20, 25.0, 'Accessoire')
(4, 'Charlie', 'Laptop', 2, 1250.0, 'High-Tech')
(5, 'Alice', 'Ecran', 4, 300.0, 'High-Tech')
(6, 'Bob', 'Webcam', 5, 45.0, 'Accessoire')
(7, 'David', 'Chaise', 10, 150.0, 'Mobilier')
(8, 'David', 'Bureau', 2, 400.0, 'Mobilier')
(9, 'Alice', 'Laptop', 5, 1200.0, 'High-Tech')
(10, 'Bob', 'Clavier', 10, 50.0, 'Accessoire')
(11, 'Alice', 'Souris', 20, 25.0, 'Accessoire')
(12, 'Charlie', 'Laptop', 2, 1250.0, 'High-Tech')
(13, 'Alice', 'Ecran', 4, 300.0, 'High-Tech')
(14, 'Bob', 'Webcam', 5, 45.0, 'Accessoire')
(15, 'David', 'Chaise', 10, 150.0, 'Mobilier')
(16, 'David', 'Bureau', 2, 400.0, 'Mobilier')


###  SELECT - Exemple 2 (Sélection ciblée)

**Cas d'usage :** On veut juste une liste des produits et leur catégorie pour un catalogue, sans voir les prix ou les vendeurs.

In [8]:
# On liste explicitement les colonnes désirées
sql_sel_2 = "SELECT produit, categorie FROM ventes"
display(pd.read_sql_query(sql_sel_2, conn))

,produit,categorie
0,Laptop,High-Tech
1,Clavier,Accessoire
2,Souris,Accessoire
3,Laptop,High-Tech
4,Ecran,High-Tech
5,Webcam,Accessoire
6,Chaise,Mobilier
7,Bureau,Mobilier
8,Laptop,High-Tech
9,Clavier,Accessoire


---

## 2. CALCULS & AGRÉGATIONS (Arithmétique SQL)

SQL n'est pas juste un moteur de recherche, c'est une calculatrice puissante qui travaille directement sur les données avant de vous les envoyer.

###  Calcul Scalaire - Exemple 1 (Opération ligne par ligne)

**Cas d'usage :** Calculer le chiffre d'affaires (CA) pour chaque ligne de vente (`quantité * prix`). Ce chiffre n'existe pas dans la base, on le crée à la volée.

In [10]:
# On crée une colonne virtuelle 'total_vente'
sql_calc_1 = """
SELECT
    vendeur,
    produit,
    quantite,
    prix_unitaire,
    (quantite * prix_unitaire) as total_vente
FROM ventes
"""
display(pd.read_sql_query(sql_calc_1, conn))

,vendeur,produit,quantite,prix_unitaire,total_vente
0,Alice,Laptop,5,1200.0,6000.0
1,Bob,Clavier,10,50.0,500.0
2,Alice,Souris,20,25.0,500.0
3,Charlie,Laptop,2,1250.0,2500.0
4,Alice,Ecran,4,300.0,1200.0
5,Bob,Webcam,5,45.0,225.0
6,David,Chaise,10,150.0,1500.0
7,David,Bureau,2,400.0,800.0
8,Alice,Laptop,5,1200.0,6000.0
9,Bob,Clavier,10,50.0,500.0


###  Fonctions d'Agrégation - Exemple 2 (Statistiques Globales)

**Cas d'usage :** Obtenir un résumé global de l'activité (CA Total, Prix moyen, Nombre de ventes).
*Note : Cela écrase les détails et ne renvoie qu'une seule ligne.*

In [11]:
sql_calc_2 = """
SELECT
    COUNT(*) as nombre_transactions,
    SUM(quantite * prix_unitaire) as chiffre_affaires_total,
    AVG(prix_unitaire) as prix_moyen_article,
    MAX(prix_unitaire) as article_le_plus_cher
FROM ventes
"""
display(pd.read_sql_query(sql_calc_2, conn))

,nombre_transactions,chiffre_affaires_total,prix_moyen_article,article_le_plus_cher
0,16,26450.0,427.5,1250.0


---

## 3. WHERE (Le Filtrage)

C'est le "garde-barrière". Il décide quelles lignes sont retenues avant même de faire des calculs ou des tris.

###  WHERE - Exemple 1 (Comparaison numérique)

**Cas d'usage :** Trouver les "grosses ventes" (celles où la quantité dépasse 5 unités).

In [14]:
sql_where_1 = """
SELECT  vendeur, produit, quantite
FROM ventes
WHERE quantite > 5
"""
display(pd.read_sql_query(sql_where_1, conn))

,vendeur,produit,quantite
0,Bob,Clavier,10
1,Alice,Souris,20
2,David,Chaise,10
3,Bob,Clavier,10
4,Alice,Souris,20
5,David,Chaise,10


###  WHERE - Exemple 2 (Comparaison textuelle et logique multiple)

**Cas d'usage :** Trouver les ventes d'Alice, MAIS UNIQUEMENT dans la catégorie 'High-Tech'.
*On utilise `AND` pour cumuler les conditions.*

In [15]:
sql_where_2 = """
SELECT * FROM ventes
WHERE vendeur = 'Alice' AND categorie = 'High-Tech'
"""
display(pd.read_sql_query(sql_where_2, conn))

,id,vendeur,produit,quantite,prix_unitaire,categorie
0,1,Alice,Laptop,5,1200.0,High-Tech
1,5,Alice,Ecran,4,300.0,High-Tech
2,9,Alice,Laptop,5,1200.0,High-Tech
3,13,Alice,Ecran,4,300.0,High-Tech


---

## 4. ORDER BY (Le Tri)

Par défaut, l'ordre des lignes en SQL n'est **jamais garanti**. Si vous voulez un ordre précis, vous devez l'exiger.

### ORDER BY - Exemple 1 (Ascendant / Alphabétique)

**Cas d'usage :** Trier les produits par catégorie pour grouper visuellement les résultats.

In [20]:
# ASC est l'option par défaut (A -> Z ou 0 -> 9)
sql_sort_1 = """
SELECT distinct categorie, produit, prix_unitaire
FROM ventes
ORDER BY categorie 
"""
display(pd.read_sql_query(sql_sort_1, conn))

,categorie,produit,prix_unitaire
0,Accessoire,Clavier,50.0
1,Accessoire,Souris,25.0
2,Accessoire,Webcam,45.0
3,High-Tech,Laptop,1200.0
4,High-Tech,Laptop,1250.0
5,High-Tech,Ecran,300.0
6,Mobilier,Chaise,150.0
7,Mobilier,Bureau,400.0


### Cellule 8 : ORDER BY - Exemple 2 (Descendant + Limite)

**Cas d'usage :** Le "Top 3" des ventes les plus chères.
*On combine un calcul, un tri décroissant (`DESC`) et une limite (`LIMIT`).*

In [24]:
sql_sort_2 = """
SELECT distinct vendeur, produit, (quantite * prix_unitaire) as montant_total
FROM ventes
ORDER BY montant_total DESC
LIMIT 5
"""
display(pd.read_sql_query(sql_sort_2, conn))

,vendeur,produit,montant_total
0,Alice,Laptop,6000.0
1,Charlie,Laptop,2500.0
2,David,Chaise,1500.0
3,Alice,Ecran,1200.0
4,David,Bureau,800.0


---

## 5. GROUP BY (Le Regroupement)

C'est l'outil roi de l'analyse (Business Intelligence). Il permet de "comprimer" les lignes qui ont un point commun pour faire des statistiques dessus.

### Cellule 9 : GROUP BY - Exemple 1 (Comptage simple)

**Cas d'usage :** Combien de ventes a réalisé chaque vendeur ?

In [25]:
sql_group_1 = """
SELECT
    vendeur,
    COUNT(*) as nombre_de_ventes
FROM ventes
GROUP BY vendeur
"""
display(pd.read_sql_query(sql_group_1, conn))

,vendeur,nombre_de_ventes
0,Alice,6
1,Bob,4
2,Charlie,2
3,David,4


### Cellule 10 : GROUP BY - Exemple 2 (Agrégation complexe multi-critères)

**Cas d'usage :** Quel est le Chiffre d'Affaires généré par Catégorie ?
*C'est la base de tout tableau de bord financier.*

In [ ]:
sql_group_2 = """
SELECT
    categorie,
    SUM(quantite) as volume_total_articles,
    SUM(quantite * prix_unitaire) as ca_total_categorie
FROM ventes
GROUP BY categorie
ORDER BY ca_total_categorie DESC
"""
display(pd.read_sql_query(sql_group_2, conn))

### Cellule 11 : Fermeture de ce module

In [ ]:
conn.close()
print("Module 'Sélection' terminé.")